# Model comparison

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression
from fair_models import calc_reweight
from fair_models import FairScalarization, EqualScalarization

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


## Parte 1: Data treatment

In [2]:
dataset = pd.read_csv("Datasets/lsat.csv")

dataset = dataset.dropna()

In [3]:
dataset.head()

,Unnamed: 0,decile1b,decile3,ID,decile1,sex,race,cluster,lsat,ugpa,...,black,hisp,pass_bar,bar,tier,index6040,indxgrp,indxgrp2,dnn_bar_pass_prediction,gpa
0,0,10.0,10.0,2,10.0,1.0,7.0,1.0,44.0,3.5,...,0,0,1.0,a Passed 1st time,4.0,886.842082,g 700+,i 820+,0.979804,3.5
1,1,5.0,4.0,3,5.0,1.0,7.0,2.0,29.0,3.5,...,0,0,1.0,a Passed 1st time,2.0,649.999987,f 640-700,f 640-700,0.979804,3.5
2,2,3.0,2.0,36,3.0,2.0,7.0,3.0,36.0,3.5,...,0,0,1.0,a Passed 1st time,3.0,760.526298,g 700+,h 760-820,0.979804,3.5
3,3,7.0,4.0,52,7.0,2.0,7.0,3.0,39.0,3.5,...,0,0,1.0,a Passed 1st time,3.0,807.894717,g 700+,h 760-820,0.979804,3.5
4,4,9.0,8.0,55,9.0,2.0,7.0,4.0,48.0,3.5,...,0,0,1.0,a Passed 1st time,5.0,949.999974,g 700+,i 820+,0.979804,3.5


In [4]:
_COLUMN_NAMES = [
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]

dataset['gender'] = dataset['gender'].astype(str)
dataset['race1'] = dataset['race1'].astype(str)
dataset = dataset[_COLUMN_NAMES]

dataset.head()

,gender,lsat,pass_bar,race1,ugpa
0,female,44.0,1.0,white,3.5
1,female,29.0,1.0,white,3.5
2,male,36.0,1.0,white,3.5
3,male,39.0,1.0,white,3.5
4,male,48.0,1.0,white,3.5


In [5]:
dataset.groupby('race1').mean()

,lsat,pass_bar,ugpa
race1,,,
asian,36.290886,0.924051,3.210886
black,29.394073,0.777963,2.890735
hisp,33.306695,0.875810,3.070518
other,34.936096,0.901070,3.137968
white,37.463003,0.967326,3.244682


In [6]:
# Convert Sex value to 0 and 1
dataset["gender"] = dataset["gender"].map({"male": 0, "female":1})
dataset["race1"] = dataset["race1"].map({"white": 0, "black":1, "asian":1, "hisp": 1, "other": 1})
dataset["pass_bar"] = dataset["pass_bar"].map({1: 1, 0:-1})

In [7]:
dataset.head()

,gender,lsat,pass_bar,race1,ugpa
0,1,44.0,1,0,3.5
1,1,29.0,1,0,3.5
2,0,36.0,1,0,3.5
3,0,39.0,1,0,3.5
4,0,48.0,1,0,3.5


In [8]:
X = dataset.drop(['pass_bar'], axis=1)

In [9]:
y = dataset['pass_bar']

In [10]:
X.shape

(20427, 4)

In [11]:
# 395, 808
random_seed = 2000#np.random.randint(0, 1000)
random_seed

2000

In [12]:
random_seed2 = 2000#np.random.randint(0, 1000)
random_seed2

2000

In [13]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=8000, random_state = random_seed)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=4000, random_state = random_seed2)

In [14]:
fair_feat = 'race1'

In [15]:
sample_weight = calc_reweight(X_train, y_train, fair_feat)

## Parte 3: Comparando os modelos

In [16]:
optimize_metrics = {'accuracy': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': [],
                'RegEqMoo': []}}

In [17]:
metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']

In [18]:
methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo', 'RegEqMoo']
index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)
results_val = pd.DataFrame(index=index, columns=metrics)

In [19]:
fair_feature = "race1"

In [20]:
# RegLog

for metric in metrics:
    reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric).tune()
    # RegMoo
    optimize_metrics[metric]['LogReg'] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]
    results_test.loc[('LogReg',metric),:] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]

In [21]:
# RegEqual

for metric in metrics:
    reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, metric=metric, base_model='equal').tune()

    optimize_metrics[metric]['RegEqual'] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]
    results_test.loc[('RegEqual',metric),:] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]

In [22]:
# RegDemo

for metric in metrics:
    reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val,fair_feature, metric=metric, base_model='demographic').tune()

    optimize_metrics[metric]['RegDemo'] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]
    results_test.loc[('RegDemo',metric),:] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/cvxpy/problems/problem.py:1055: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


In [23]:
# RegRewe

for metric in metrics:
    reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, fair_feature,metric=metric, sample_weight=sample_weight).tune()

    optimize_metrics[metric]['RegRewe'] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]
    results_test.loc[('RegRewe',metric),:] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]

In [24]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_moo = moo_learn.tune(metric)
    
    results_test.loc[('RegMoo',metric),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

Using license file /opt/gurobi/gurobi.lic
Academic license - for non-commercial use only


/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: Runtim

In [25]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)
moo_learn.tune()

for metric in metrics:
    reg_eqmoo = moo_learn.tune(metric)
    
    results_test.loc[('RegEqMoo',metric),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for race1 == 1, returning 0
  RuntimeWarning,
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/root/anaconda3/envs/py36/lib/python3.6/site-packages/sklego/metrics.py:79: Runtim

In [26]:
results_test

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy           0.948875          0.991589  0.980174    0.297658
         equal_opportunity     0.948                 1         1    0.298473
         p_percent             0.948                 1         1    0.298473
         c_variation          0.9485          0.999065  0.995242    0.297403
RegEqual accuracy           0.948375                 1  0.995242     0.29756
         equal_opportunity     0.948                 1  0.995242    0.298473
         p_percent             0.948                 1  0.995242    0.298473
         c_variation           0.948                 1  0.995242    0.298473
RegDemo  accuracy              0.948                 1         1    0.298473
         equal_opportunity     0.948                 1         1    0.298473
         p_percent             0.948                 1         1    0.298473
         c_variation          0.9485          0.999065  0.995242    0.297403
RegRewe  accuracy              0.948                 1         1    0.298473
         equal_opportunity     0.948                 1         1    0.298473
         p_percent             0.948                 1         1    0.298473
         c_variation           0.948                 1         1    0.298473
RegMoo   accuracy           0.948375          0.999065  0.996035    0.297709
         equal_opportunity     0.948                 1         1    0.298473
         p_percent             0.948                 1         1    0.298473
         c_variation        0.948375          0.999065  0.996035    0.297709
RegEqMoo accuracy           0.948875          0.991589  0.980174    0.297658
         equal_opportunity     0.948                 1         1    0.298473
         p_percent             0.948                 1         1    0.298473
         c_variation        0.948625          0.998131  0.992863    0.297245

## Models with ensemble

In [27]:
ensemble_metrics = {'accuracy': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'equal_opportunity': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'p_percent': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []},
                'c_variation': {'ensembles': ['voting soft', 'voting hard', 'knorau', 'knorae'],
                'RegMoo': [],
                'RegEqMoo': []}}

In [28]:
ensembles = ['voting soft', 'voting hard', 'knorau', 'knorae']

In [29]:
methods = ['RegMoo', 'RegEqMoo']
index = [sum([[method]*len(ensembles) for method in methods],[]),
        sum([ensembles for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)

In [30]:
# RegMoo

scalarization = FairScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)

for ensemble in ensembles:
    reg_moo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegMoo',ensemble),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

In [31]:
# RegEqMoo

scalarization = EqualScalarization(X_train, y_train, fair_feature)
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val, fair_feature, scalarization)

for ensemble in ensembles:
    reg_eqmoo = moo_learn.ensemble_model(ensemble)
    
    results_test.loc[('RegEqMoo',ensemble),:] = [reg_eqmoo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column=fair_feature)(reg_eqmoo, X_test, y_test),
                            p_percent_score(sensitive_column=fair_feature)(reg_eqmoo, X_test),
                            coefficient_of_variation(reg_eqmoo, X_test, y_test)]

In [32]:
results_test

accuracy equal_opportunity p_percent c_variation
RegMoo   voting soft   0.94825                 1  0.998414    0.297865
         voting hard    0.9485          0.998131  0.993656    0.297552
         knorau       0.948625          0.999065  0.994449    0.297097
         knorae       0.911125          0.896862  0.882772    0.420914
RegEqMoo voting soft     0.948                 1         1    0.298473
         voting hard     0.948                 1         1    0.298473
         knorau          0.948                 1         1    0.298473
         knorae          0.911          0.896862  0.883592    0.421156